In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [ ]:
%%bash

cp drive/MyDrive/Colab\ Notebooks/data-files/dogs-vs-cats.zip sample_data/
unzip sample_data/dogs-vs-cats.zip -d sample_data
unzip sample_data/dogs-vs-cats/train.zip -d sample_data/dogs-vs-cats
unzip sample_data/dogs-vs-cats/test1.zip -d sample_data/dogs-vs-cats
mv sample_data/dogs-vs-cats/test1 sample_data/dogs-vs-cats/test
rm -rf sample_data/__MACOSX

In [3]:
import os
import shutil

src_base = 'sample_data/dogs-vs-cats'
dest_base = 'sample_data/dogs-vs-cats2'

if os.path.exists(dest_base): # 경로 존재여부 확인
  shutil.rmtree(dest_base) # train 경로 및 하위 경로 삭제
os.mkdir(dest_base) # 디렉터리 만들기

# train 폴더의 0 ~ 1000 : train, 1000 ~ 1500 : validation, 1500 ~ 2000 : test 세트로 구성
for start, stop, path in zip([0, 1000, 1500], [1000, 1500, 2000], ['train', 'validation', 'test']):
  os.mkdir(os.path.join(dest_base, path))
  for sub_path in ['cat', 'dog']:
    os.mkdir(os.path.join(dest_base, path, sub_path))
    for idx in np.arange(start, stop):
      fname = f'{sub_path}.{idx}.jpg'
      shutil.copy(os.path.join(src_base, 'train', fname), os.path.join(dest_base, path, sub_path, fname)) # 파일 복사

In [5]:
# 입력 자동화 도구 만들기 : 파일을 읽어서 모델에 입력 가능한 형식으로 변환하는 도구

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale=1/255) # 특정 디렉터리의 파일을 읽어서 모델의 입력데이터로 변환하는 도구
validation_data_generator = ImageDataGenerator(rescale=1/255)
test_data_generator = ImageDataGenerator(rescale=1/255)

In [6]:
# generator를 사용해서 파일 데이터 읽기
datasets = []
for path, generator in zip(['train', 'validation', 'test'],
                           [train_data_generator, validation_data_generator, test_data_generator]):
  dataset = generator.flow_from_directory(directory=f"sample_data/dogs-vs-cats2/{path}",
                                          target_size=(256, 256),
                                          batch_size=32,
                                          class_mode="binary")
  datasets.append(dataset)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 이미 학습 완료된 비슷한 모델 활용 ( 전이학습 )

# 1. 다른 모델의 합성곱 층이 출력한 데이터를 현재 모델의 입력으로 사용
# 2. 다른 모델과 현재 모델 결합 1 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층은 학습하지 않음 )
# 3. 다른 모델과 현재 모델 결합 2 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층의 일부는 학습 )

In [7]:
# 사전 학습 모델 준비

base_model = tf_keras.applications.vgg16.VGG16(include_top=False,
                                               weights='imagenet',
                                               input_shape=(256, 256, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
# CNN 모델 훈련

model = tf_keras.Sequential([
    tf_keras.layers.Input(shape=(256, 256, 3)),
    # 합성곱 층 ( 이미지의 특성 읽는 층 )
    tf_keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"),
    tf_keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf_keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
    tf_keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf_keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"),
    tf_keras.layers.MaxPool2D(pool_size=(2, 2)),

    # 완전 연결 층 ( 이미지를 판별하는 층 )
    tf_keras.layers.Flatten(),
    tf_keras.layers.Dense(units=512, activation='relu'),
    tf_keras.layers.Dense(units=1, activation='sigmoid')
])

In [ ]:
model.summary()
# tf_keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [ ]:
history = model.fit(datasets[0],
                    steps_per_epoch=60, epochs=20,
                    validation_data=datasets[1], validation_steps=30)

Epoch 1/20
60/60 [==============================] - 10s 164ms/step - loss: 0.6071 - accuracy: 0.6859 - val_loss: 0.6350 - val_accuracy: 0.6438
Epoch 2/20
60/60 [==============================] - 8s 129ms/step - loss: 0.5176 - accuracy: 0.7516 - val_loss: 0.6225 - val_accuracy: 0.6906
Epoch 3/20
60/60 [==============================] - 9s 155ms/step - loss: 0.4104 - accuracy: 0.8167 - val_loss: 0.6097 - val_accuracy: 0.6812
Epoch 4/20
60/60 [==============================] - 10s 174ms/step - loss: 0.2740 - accuracy: 0.8845 - val_loss: 0.7138 - val_accuracy: 0.7188
Epoch 5/20
60/60 [==============================] - 8s 135ms/step - loss: 0.1510 - accuracy: 0.9433 - val_loss: 0.8842 - val_accuracy: 0.7094
Epoch 6/20
60/60 [==============================] - 9s 153ms/step - loss: 0.0935 - accuracy: 0.9674 - val_loss: 1.1393 - val_accuracy: 0.6854
Epoch 7/20
60/60 [==============================] - 10s 158ms/step - loss: 0.0433 - accuracy: 0.9884 - val_loss: 1.4016 - val_accuracy: 0.6948
Epo